```{contents}
```
## Logging


**Logging** is the practice of **recording runtime events** (messages, errors, timings, states) so you can **debug, monitor, and audit** an application.

In LLM systems, logging typically captures:

* Inputs and outputs
* Errors and exceptions
* Latency and timing
* Tool usage
* Application state

Logging is **developer-controlled** and **flat**, unlike tracing (which is hierarchical and automatic).

---

### Why Logging Is Important in LLM Systems

Logging helps you answer:

* *Did the request reach the model?*
* *What input was sent?*
* *What output was returned?*
* *Did an error occur?*
* *How long did it take?*

It is essential for:

* Debugging production issues
* Compliance and audits
* Incident investigation
* Operational monitoring

---

### Logging vs Tracing (Quick Contrast)

| Aspect    | Logging          | Tracing            |
| --------- | ---------------- | ------------------ |
| Structure | Flat text/events | Hierarchical       |
| Control   | Manual           | Automatic          |
| Scope     | App-level        | Execution-level    |
| Storage   | Files / stdout   | Tracing backend    |
| Use case  | Debug & audit    | Deep observability |

Both are **complementary**, not competing.

---

### Basic Logging in Python (Foundation)

```python
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

logging.info("Application started")
logging.warning("This is a warning")
logging.error("Something went wrong")
```

This sets up:

* Log level
* Timestamped output
* Console logging

---

### Logging in an LLM Pipeline (LangChain Example)

Using LangChain.

```python
import logging
from langchain_openai import ChatOpenAI

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

llm = ChatOpenAI()

def ask_llm(question: str):
    logger.info("LLM request received: %s", question)

    response = llm.invoke(question)

    logger.info("LLM response length: %d", len(response.content))
    return response.content
```

What gets logged:

* User input
* Response metadata (not full text, safer)
* Execution flow

---

### Logging Errors and Exceptions

```python
try:
    result = ask_llm("Explain logging")
except Exception as e:
    logging.exception("LLM call failed")
```

`logging.exception()` automatically logs:

* Error message
* Full stack trace

---

### Logging Inside Runnable Chains

```python
from langchain_core.runnables import RunnableLambda

def log_input(x):
    logging.info("Runnable input: %s", x)
    return x

chain = (
    RunnableLambda(log_input)
    | RunnableLambda(lambda x: x.upper())
)
```

Each runnable can log:

* Inputs
* Outputs
* Decisions

---

### Logging with Callback Handlers (Advanced Pattern)

```python
from langchain.callbacks.base import BaseCallbackHandler
import logging

class LoggingCallback(BaseCallbackHandler):
    def on_chain_start(self, serialized, inputs, **kwargs):
        logging.info("Chain started with inputs: %s", inputs)

    def on_llm_end(self, response, **kwargs):
        logging.info("LLM finished")

llm = ChatOpenAI(callbacks=[LoggingCallback()])
llm.invoke("Explain logging")
```

This logs **internal execution events**.

---

### Logging in FastAPI (Real-World)

```python
from fastapi import FastAPI
import logging

app = FastAPI()
logger = logging.getLogger("api")

@app.get("/chat")
async def chat(q: str):
    logger.info("Incoming query: %s", q)
    return {"answer": q.upper()}
```

Typical production flow:

* Request received
* Processing steps logged
* Response returned

---

### Log Levels (Use Correctly)

| Level    | Use                    |
| -------- | ---------------------- |
| DEBUG    | Detailed dev info      |
| INFO     | Normal operations      |
| WARNING  | Unexpected but handled |
| ERROR    | Failed operation       |
| CRITICAL | App may crash          |

Example:

```python
logging.debug("Prompt template rendered")
logging.info("LLM call successful")
logging.error("Timeout calling model")
```

---

### What NOT to Log

Avoid logging:

* API keys
* Tokens
* PII (emails, phone numbers)
* Full prompts/responses in prod

Safe pattern:

```python
logging.info("LLM called, tokens=%d", token_count)
```

---

### Logging to Files (Production)

```python
logging.basicConfig(
    filename="app.log",
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)
```

Often combined with:

* Log rotation
* Centralized log collectors (ELK, CloudWatch)

---

### Common Logging Mistakes

* Logging too much (noise)
* Logging secrets
* Using `print()` instead of logging
* No log levels
* No timestamps

---

### Mental Model

Logging is your **black box recorder**:

```
Event happens → log entry written → stored for later inspection
```

Tracing tells *how execution flowed*
Logging tells *what happened at key points*

---

### Key Takeaways

* Logging records **application events**
* Manual and developer-controlled
* Essential for debugging and audits
* Complements tracing and callbacks
* Must be secure and level-based
